初始化

In [ ]:
# 掛載雲端
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 環境初始化 (大約三至五分鐘)
! wget -O init_env.sh https://www.dropbox.com/s/4jvf5hg8paip699/init_env%20spark%203.1.1.sh?dl=0 && \
bash init_env.sh

import os, sys
os.environ['SPARK_HOME'] = "/usr/local/spark"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.9-src.zip")
from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext()

#my env
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession(sc)

Wiki資料讀取

In [ ]:
wiki_rdd = sc.textFile("drive/My Drive/wiki.plain.text.line.spark.rdd.format")

詞頻計算，將結果存到word_count.csv

In [ ]:
word_counts_RDD = wiki_rdd\
          .flatMap(lambda line: line.split(" "))\
          .filter(lambda word: len(word)>1 and not word.isnumeric() )\
          .map(lambda word: (word, 1))\
          .reduceByKey(lambda a, b: a + b)\
          .sortBy(lambda x:x[1], ascending=False)
          
word_count_DF = word_counts_RDD.toDF(("word", "count")).toPandas()  
word_count_DF.to_csv("drive/My Drive/word_count.csv", index=False)

整理【關鍵字與出現此字的文章列表】，將結果存到word_article.csv

In [ ]:
from typing import Iterable
from typing import List

def mark_and_split(article: str) -> List[tuple]:
  article_split = article.split()
  id = article_split[0]
  content  = article_split[1:]
  return [(word, id) for word in content]

def To_list_like_string(input: Iterable[str]) -> str:
  """ 
  Turn input Iterable[str] into a【no repeated element】list,
  than turn the list into a【list expression】string.

  Example_input : list[str] = ['100', '101', '104', '101']
  Example_output : str = '[100,101,104]'
  """
  input = list(set(input))
  output = ['[']
  for i in input:
    output.append(i)
    output.append(',')
  output[-1] = ']'
  return ''.join(output)

word_group_RDD = wiki_rdd\
          .flatMap(mark_and_split)\
          .filter(lambda pair: len(pair[0])>1)\
          .groupByKey()\
          .map(lambda pair: (pair[0],To_list_like_string(pair[1])))

word_group_DF = word_group_RDD.toDF(("word", "id"))
word_group_DF.coalesce(1).write.csv("drive/My Drive/word_article.csv")